In [1]:
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def KNNPCA(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
       	
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
        
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
        	
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Ka)
    trained_model=clf.fit(X_train,z_train)
    trained_model.fit(X_train,z_train )
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

############ Happy And Sad########################
def KNNPCAHS(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]


############ All Emotions########################33
def KNNPCACombined(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

################ Run Each Method Multiple Times #####################
def Run100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        KNN=KNNPCA(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        arousal+=[KNN[0]]
        valence+=[KNN[1]]
        F1_ScoreArousal+=[KNN[2]]
        F1_ScoreValence+=[KNN[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=KNNPCACombined(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=KNNPCAHS(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

##################Main#########################
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,33):
        print("s"+str(i))
        #Happy and Sad
        if not ("s"+str(i) == "s31"):
            var =RunHS100("s"+str(i))
            print("Happy Sad")
            print(var)
            HSAccurecy+=var[0]
            HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for 31 users "+str(HSAccurecy/31))
    print("F1 happy sad for  31 users "+str(HSF1/31))
    print("Accurecy Combined sad for 32 users"+str(CombinedAccurecy/32))
    print("F1 Combined sad for  32 users "+str(CombinedF1/32))
    print("Accurecy Valence for  32 users "+str(ValenceAccurecy/32))
    print("F1 Valence for 32 users "+str(ValenceF1/32))
    print("Accurecy Arousal for  32 users "+str(ArousalAccurecy/32))
    print("F1 Arousal for  32 users "+str(ArousalF1/32))


s01
Array of average Happy Sad accurecies for user s01
Counter({40.0: 45, 60.0: 26, 20.0: 15, 80.0: 10, 100.0: 2, 0.0: 2})
Happy Sad
[46.6, 0.44268253968253973]
Array of average All Emotions accurecies for user s01
Counter({25.0: 32, 37.5: 22, 50.0: 21, 12.5: 12, 62.5: 8, 0.0: 4, 75.0: 1})
Combined Emotions
[34.0, 0.2985560064935065]
Arousal 56.0
Counter({50.0: 39, 62.5: 28, 75.0: 15, 37.5: 11, 25.0: 4, 87.5: 3})
Valence 50.75
Counter({50.0: 25, 62.5: 24, 37.5: 20, 75.0: 13, 25.0: 11, 12.5: 4, 87.5: 2, 100.0: 1})
Arousal Valence
[56.0, 50.75, 0.5069542124542123, 0.49108510933510935]
s02
Array of average Happy Sad accurecies for user s02
Counter({80.0: 47, 60.0: 28, 100.0: 21, 40.0: 4})
Happy Sad
[77.0, 0.7421031746031745]
Array of average All Emotions accurecies for user s02
Counter({50.0: 31, 37.5: 25, 62.5: 20, 25.0: 12, 75.0: 7, 12.5: 5})
Combined Emotions
[46.25, 0.39571542346542343]
Arousal 62.5
Counter({62.5: 37, 75.0: 24, 50.0: 20, 87.5: 8, 37.5: 7, 25.0: 3, 100.0: 1})
Valence 6

Array of average Happy Sad accurecies for user s15
Counter({50.0: 53, 75.0: 22, 25.0: 18, 100.0: 6, 0.0: 1})
Happy Sad
[53.5, 0.5131904761904762]
Array of average All Emotions accurecies for user s15
Counter({25.0: 33, 37.5: 25, 50.0: 17, 12.5: 14, 0.0: 5, 62.5: 5, 75.0: 1})
Combined Emotions
[31.75, 0.30129464285714286]
Arousal 46.375
Counter({50.0: 32, 37.5: 31, 25.0: 14, 62.5: 13, 75.0: 7, 87.5: 2, 12.5: 1})
Valence 63.625
Counter({62.5: 31, 50.0: 22, 75.0: 21, 87.5: 13, 37.5: 11, 100.0: 2})
Arousal Valence
[46.375, 63.625, 0.4490881340881341, 0.6357479464979464]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 57, 100.0: 24, 60.0: 16, 40.0: 3})
Happy Sad
[80.4, 0.7953809523809524]
Array of average All Emotions accurecies for user s16
Counter({50.0: 29, 37.5: 27, 62.5: 18, 25.0: 18, 12.5: 6, 75.0: 1, 0.0: 1})
Combined Emotions
[41.875, 0.36941910173160175]
Arousal 69.25
Counter({75.0: 33, 62.5: 32, 87.5: 16, 50.0: 10, 100.0: 4, 25.0: 3, 37.5: 2})
Valence 62.25
C

Array of average Happy Sad accurecies for user s29
Counter({66.66666666666666: 38, 83.33333333333334: 28, 50.0: 23, 33.33333333333333: 6, 100.0: 5})
Happy Sad
[67.16666666666666, 0.6808055555555554]
Array of average All Emotions accurecies for user s29
Counter({50.0: 32, 62.5: 26, 37.5: 23, 25.0: 9, 87.5: 5, 75.0: 5})
Combined Emotions
[51.25, 0.46889028332778326]
Arousal 73.75
Counter({75.0: 38, 87.5: 22, 62.5: 19, 100.0: 9, 50.0: 6, 37.5: 5, 25.0: 1})
Valence 60.875
Counter({62.5: 36, 75.0: 26, 50.0: 22, 37.5: 11, 87.5: 4, 25.0: 1})
Arousal Valence
[73.75, 60.875, 0.7049906482406482, 0.6043037518037518]
s30
Array of average Happy Sad accurecies for user s30
Counter({57.14285714285714: 44, 42.857142857142854: 22, 71.42857142857143: 20, 85.71428571428571: 7, 28.57142857142857: 6, 14.285714285714285: 1})
Happy Sad
[56.71428571428571, 0.5438742368742369]
Array of average All Emotions accurecies for user s30
Counter({50.0: 36, 37.5: 32, 62.5: 11, 25.0: 10, 75.0: 6, 12.5: 4, 87.5: 1})
Comb